#Instalar as bibliotecas

In [2]:
!pip install google-cloud-bigquery

In [3]:
!pip install joblib

In [4]:
import google.auth
from google.colab import auth
from google.cloud import bigquery
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import joblib
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

ModuleNotFoundError: No module named 'google.colab'

#Tratamento da base

In [ ]:

# Autenticar o usuário
auth.authenticate_user()

# Inicialize um cliente do BigQuery
clientbq = bigquery.Client(project='aulafiap-403000')


In [ ]:
# Consulta SQL para selecionar os dados da tabela
query = """
    SELECT *
    FROM emf-teacher.trabalho_loans.loan_default
"""

query_job = clientbq.query(query)
dados = query_job.to_dataframe()


In [ ]:
dados.columns

In [ ]:
dados.head(
)

In [ ]:
#Exibir a quantidade de nulos existentes
dados.isnull().sum()

In [ ]:
# dados em porcentagem de nulos
((dados.isnull().sum()/dados.shape[0]) * 100)

In [ ]:
dados_filtrados = dados.copy()[['open_credit','Credit_Worthiness','loan_purpose','Gender','loan_type','loan_limit','credit_type','income']]

In [ ]:
((dados_filtrados.isnull().sum() / dados_filtrados.shape[0]) * 100)

In [ ]:
dados_filtrados[['loan_purpose','loan_limit','income']]

In [ ]:
dados_filtrados.info()

In [ ]:
dados_filtrados['loan_purpose'].unique()

In [ ]:
dados_filtrados['loan_limit'].unique()

In [ ]:
#Tratamento dos dados vazios
dados_filtrados.loc[:, 'loan_purpose'] = dados_filtrados.loc[:, 'loan_purpose'].fillna('unknown')

In [ ]:
#Tratamento dos dados vazios
dados_filtrados.loc[:, 'loan_limit'] = dados_filtrados.loc[:, 'loan_limit'].fillna('unknown')

In [ ]:
# Calcular a mediana de cada coluna
medianas = dados_filtrados['income'].median()

# Preencher campos vazios com a mediana de cada coluna
dados_filtrados['income'] = dados_filtrados['income'].fillna(medianas)

# Exibir o DataFrame preenchido
print(dados_filtrados['income'])

In [ ]:
((dados_filtrados.isnull().sum() / dados_filtrados.shape[0]) * 100)

In [ ]:
dados_filtrados.info()

#Feature engineering

In [ ]:
objetos = dados_filtrados.select_dtypes(include=['object'])
objetos = objetos.drop('credit_type',axis=1)


In [ ]:
objetos

In [ ]:
objetos.info()

In [ ]:
le = LabelEncoder()
# Inicialize um DataFrame vazio para armazenar os objetos transformados
objetos_transformados = pd.DataFrame()

for coluna in objetos.columns:
    # Aplique o LabelEncoder em cada coluna e adicione ao DataFrame objetos_transformados
    objetos_transformados[coluna] = le.fit_transform(objetos[coluna])



#Criação do set de feautures e Target

In [ ]:
X= pd.concat([objetos_transformados,dados_filtrados['income']],axis=1)

converter_target = {'EQUI': 1 , 'CRIF': 2, 'CIB': 3, 'EXP': 4}
y= dados_filtrados['credit_type'].map(converter_target)


#Treinar Cluster

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Criar cluster KNN
knn = KNeighborsClassifier(n_neighbors = 5)
# Treinar KNN
knn.fit(X_train,y_train)
y_pred = knn.predict(X_test)

In [ ]:
knn.predict(X_test)[0:5]

#Salvar modelo

In [ ]:
joblib.dump(knn, 'cluster_knn.pkl')

#Metricas do modelo

In [ ]:
# Calcular o RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Calcular o R²
r2 = r2_score(y_test, y_pred)

print(f'RMSE: {rmse}')
print(f'R²: {r2}')